In [1]:
class UnionTree:
  def __init__(self, size):
    self.parent = list(range(size))               # 각 노드의 부모를 저장하는 리스트 (루트 관리용)
    self.sizes = [1 for _ in range(size)]         # 루트가 관리하는 집합의 크기 (Small-to-Large 병합용)
    self.weights = [0 for _ in range(size)]       # 루트까지의 누적 가중치 (가중치 차이 쿼리용)
    self.colors = [set([i]) for i in range(size)] # 각 노드가 포함된 색상 집합 (병합 시 색상 유지용)

  # 루트 노드 찾기 + 경로 압축 + 누적 가중치 계산
  def find(self, x):
    px = self.parent[x]
    if px != x:
      self.parent[x] = self.find(px)       # 경로 압축: x의 부모를 직접 루트로 변경
      self.weights[x] += self.weights[px]  # 누적 가중치 계산: x → 루트까지의 거리
    return self.parent[x]

  # 두 노드를 하나의 집합으로 병합 (가중치 포함)
  def union(self, x, y, w=0):
    root_x = self.find(x)
    root_y = self.find(y)

    # 이미 같은 집합에 속한 경우 → 병합 불필요
    if root_x == root_y:
      return

    # root_x 집합의 크기가 작을 경우 → swap (Small to Large)
    # 목적: 항상 작은 집합을 큰 집합에 병합 (시간 최적화)
    if self.sizes[root_x] < self.sizes[root_y]:
      root_x, root_y = root_y, root_x
      w = -w

    # 병합 수행
    self.parent[root_y] = root_x
    self.weights[root_y] = self.weights[x] - self.weights[y] + w
    self.sizes[root_x] += self.sizes[root_y]

    # 색상 정보 병합: 루트 x에 색상 통합
    self.colors[root_x].update(self.colors[root_y])
    self.colors[root_y].clear()

  # 두 노드 간의 누적 가중치 차이 계산
  def get_weight_diff(self, x, y):
    if self.find(x) != self.find(y):
      return "Not connected"
    return self.weights[y] - self.weights[x]

  # 특정 노드가 속한 집합의 색상 목록 반환
  def get_colors(self, x):
    return self.colors[self.find(x)]

  # 디버깅 용도로 전체 구조 출력
  def get_structure(self):
    print("Node | Parent | Size | Weight | Colors")
    for i in range(len(self.parent)):
      print(f"{i:4} | {self.parent[i]:6} | {self.sizes[i]:4} | {self.weights[i]:6} | {sorted(self.colors[i])}")


In [2]:
# 노드 개수: 6
ut = UnionTree(6)

# 0 - 1 연결 (가중치 2)
ut.union(0, 1, 2)

# 2 - 3 연결 (가중치 4)
ut.union(2, 3, 4)

# 1 - 2 연결 (가중치 3) → 이제 0-1-2-3 모두 같은 집합
ut.union(1, 2, 3)

# 색상 확인
print("색상 (노드 3 기준):", ut.get_colors(3))  # → {0, 1, 2, 3}

# 가중치 차이 확인
print("가중치 차이 (0 → 3):", ut.get_weight_diff(0, 3))  # 0→1(2) + 1→2(3) + 2→3(4) = 9

# 구조 출력
ut.get_structure()


색상 (노드 3 기준): {0, 1, 2, 3}
가중치 차이 (0 → 3): 9
Node | Parent | Size | Weight | Colors
   0 |      0 |    4 |      0 | [0, 1, 2, 3]
   1 |      0 |    1 |      2 | []
   2 |      0 |    2 |      5 | []
   3 |      0 |    1 |      9 | []
   4 |      4 |    1 |      0 | [4]
   5 |      5 |    1 |      0 | [5]
